# Debug get_aerodynamic_center method

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from lltdrek import Wing, FlightCondition, WingPool, Simulation, PostProcessing, load_folder

airfoils_data, _ = load_folder("../sample_airfoils")
# print(airfoils_data)

wing = Wing(
    spans=[1, 1],
    chords=[1, 1, 1],
    offsets=[0, 0, 0],
    twist_angles=[0, 0, 0],
    dihedral_angles=[0, 0, 0],
    airfoils=["NACA4424", "NACA4424", "NACA4412"],
    surface_name="wing",
    N_panels=4,
    distribution_type="cosine",
    sweep_check=False,
)
wing.generate_mesh()  # Generate wing simulation elements

eh = Wing(
    spans=[1, 1],
    chords=[1, 1, 1],
    offsets=[0, 0, 0],
    twist_angles=[0, 0, 0],
    dihedral_angles=[0, 0, 0],
    airfoils=["NACA4424", "NACA4424", "NACA4412"],
    surface_name="eh",
    N_panels=4,
    distribution_type="cosine",
    sweep_check=False,
    x_pos=5
)
eh.generate_mesh()  # Generate wing simulation elements

flight_condition = FlightCondition(
    V_inf=60,
    nu=1.5e-5,
    rho=1.225,
    angles_of_attack=[5, 6, 7, 8],
    h=0,
    ground_effect_check=False,
    angular_velocity=[0, 0, 0]
)
wing.setup_airfoil_data(flight_condition, airfoils_data)  # Assign airfoil data to wing
eh.setup_airfoil_data(flight_condition, airfoils_data)

# Link wing that will be simulated with flight condition
wing_pool = WingPool(wing_list=[wing], flight_condition=flight_condition, moment_ref=[0.5, 0, 0])

simulation = Simulation(
    damping_factor = 0.5,
    max_iter = 100,
    max_residual = 1e-4,
    linear_check = False,
    show_logs = True,
    simulation_mode = "linear_first" # Solves linear version of equations before doing non linear simulation to speed up process
)

# Run simulation

simulation_results = simulation.run(wing_pool)

Running simulation for angles between 5 and 8
Linear simulation check: False
Found solution for angle 5
number of iterations: 10
Found solution for angle 6
number of iterations: 10
Found solution for angle 7
number of iterations: 9
Found solution for angle 8
number of iterations: 9


In [5]:
post_processing = PostProcessing()
aerodynamic_center = post_processing.get_aerodynamic_center(wing_pool, simulation_results)